# ESTANDARIZACIÓN DE COLONIAS*

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos

In [2]:
#directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
#archivo_filename = "../ciudades/puebla/lean/agosto_2024.csv"
#archivo_path = os.path.join(directorio_actual, archivo_filename) 
#df = pd.read_excel(archivo_path, sheet_name='Hoja3') 
df = pd.read_csv("C:/xampp/htdocs/datalpine/resources/db/Scrapining/ciudades/puebla/clean/agosto_2024.csv")
#df = pd.read_csv("D:/yoe11/Documents/TESEO/Modelo Factible/basebase.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3476 non-null   int64  
 1   categoria                  3235 non-null   object 
 2   precio                     3476 non-null   float64
 3   precio_mxn                 3476 non-null   float64
 4   precio_usd                 3476 non-null   int64  
 5   fecha_conversion           3476 non-null   object 
 6   propiedad                  3476 non-null   object 
 7   metros_total               3476 non-null   int64  
 8   metros_construido          3476 non-null   int64  
 9   precio_m2_terreno          3476 non-null   float64
 10  precio_m2_construido       3476 non-null   float64
 11  tiempo_de_publicacion      3476 non-null   object 
 12  meses_transcurridos        3476 non-null   int64  
 13  meses_transcurridos_fecha  3476 non-null   objec

# Separación

In [3]:
# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = location.split(',')
    parts += [None] * (3 - len(parts))
    return parts[:3] 
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiando valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

,ubicacion,Colonia,Municipio,Estado
0,"Puebla Centro, Puebla, Puebla",Puebla Centro,Puebla,Puebla
1,"Las Hadas Mundial 86, Puebla, Puebla",Las Hadas Mundial 86,Puebla,Puebla
2,"Plazas de Amalucan, Puebla, Puebla",Plazas de Amalucan,Puebla,Puebla
3,"Héroes de Puebla, Puebla, Puebla",Héroes de Puebla,Puebla,Puebla
4,"Plazas de Amalucan, Puebla, Puebla",Plazas de Amalucan,Puebla,Puebla
...,...,...,...,...
3471,"Puebla, Puebla, Puebla",Puebla,Puebla,Puebla
3472,"Angelópolis, Puebla, Puebla",Angelópolis,Puebla,Puebla
3473,"La Paz, Puebla, Puebla",La Paz,Puebla,Puebla
3474,"San José Mayorazgo, Puebla, Puebla",San José Mayorazgo,Puebla,Puebla


### Estandarizar

In [4]:
# Minúsculas
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Estado'] = df['Estado'].str.strip().str.lower()

In [5]:
# Esta línea de código aplica la función unicodedata.normalize a la columna 'Colonia' del DataFrame df.
# El argumento 'NFKD' indica que se debe aplicar la normalización NFKD, que elimina los acentos y otros caracteres diacríticos.
#df['Colonia'] = df['Colonia'].apply(lambda x: unicodedata.normalize('NFKD', x))

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)


### Separación de tipo de lugar, colonias

In [6]:
# Añade las diferentes formas de identificar el tipo de lugar (etiquetas de ubicación)
tipos = ['condominio', 'fraccionamiento', 'residencial', 'colonia', 'fracc', 'uhp', 'urbanización','urbanizacion', 'sector', 'barrio', 'zona', 'sector', 'unidad habitacional', 'unidad residencial']
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['tipo_lugar'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['tipo_lugar'], '').strip() if row['tipo_lugar'] else row['Colonia'], axis=1)
print(df[['Colonia','tipo_lugar','ubicacion']])

                   Colonia tipo_lugar                             ubicacion
0            puebla centro       None         Puebla Centro, Puebla, Puebla
1     las hadas mundial 86       None  Las Hadas Mundial 86, Puebla, Puebla
2       plazas de amalucan       None    Plazas de Amalucan, Puebla, Puebla
3         heroes de puebla       None      Héroes de Puebla, Puebla, Puebla
4       plazas de amalucan       None    Plazas de Amalucan, Puebla, Puebla
...                    ...        ...                                   ...
3471                puebla       None                Puebla, Puebla, Puebla
3472           angelopolis       None           Angelópolis, Puebla, Puebla
3473                la paz       None                La Paz, Puebla, Puebla
3474    san jose mayorazgo       None    San José Mayorazgo, Puebla, Puebla
3475      centro historico       None      Centro Histórico, Puebla, Puebla

[3476 rows x 3 columns]


In [7]:
df['tipo_lugar'].unique()

array([None, 'residencial', 'barrio', 'zona'], dtype=object)

In [8]:
df['Colonia'].unique()

array(['puebla centro', 'las hadas mundial 86', 'plazas de amalucan',
       'heroes de puebla', 'san jose mayorazgo', 'ex-hacienda mayorazgo',
       'la calera', 'el carmen', 'santa maria', 'puebla', 'santa barbara',
       'san isidro castillotla', 'la loma', 'santa maria la ribera',
       'villa las flores', 'el cerrito', 'la joya',
       'san diego castillotla', 'santa clara la venta',
       'san miguel mayorazgo infonavit', 'jardines de san manuel',
       'loma linda', 'bosques del pilar', 'valle del sol',
       'bosques de chapultepec', 'club de golf puebla',
       'granjas san isidro', 'san angel', 'granjas del sur',
       'villa frontera', 'banco de puebla', 'bosques de san sebastian',
       'cuauhtemoc', 'los cipreses', 'aquiles serdan',
       'unidad satelite magisterial', 'san bartolo infonavit',
       'lomas del valle', 'villa carmel', 'las brisas', 'las palmas',
       'revolucion mexicana', 'la hacienda', 'rivera de santiago',
       'los heroes', 'geo villas l

### Corregir por similitud

In [9]:
archivo_excel = 'D:/yoe11/Documents/TESEO/Modelo Factible/Scraplining/CPdescarga.xlsx'
hojas_excel = pd.ExcelFile(archivo_excel, engine='openpyxl')
# Imprimir los nombres de las hojas
print(hojas_excel.sheet_names)

['Nota', 'Aguascalientes', 'Baja_California', 'Baja_California_Sur', 'Campeche', 'Coahuila_de_Zaragoza', 'Colima', 'Chiapas', 'Chihuahua', 'Distrito_Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán_de_Ocampo', 'Morelos', 'Nayarit', 'Nuevo_León', 'Querétaro', 'Oaxaca', 'Puebla', 'Quintana_Roo', 'Sonora', 'San_Luis_Potosí', 'Sinaloa', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz_de_Ignacio_de_la_Llave', 'Yucatán', 'Zacatecas']


In [10]:
hoja_especifica = pd.read_excel(archivo_excel, sheet_name='Puebla', engine='openpyxl')
hoja_especifica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5889 entries, 0 to 5888
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   d_codigo          5889 non-null   int64  
 1   d_asenta          5889 non-null   object 
 2   d_tipo_asenta     5889 non-null   object 
 3   D_mnpio           5889 non-null   object 
 4   d_estado          5889 non-null   object 
 5   d_ciudad          2120 non-null   object 
 6   d_CP              5889 non-null   int64  
 7   c_estado          5889 non-null   int64  
 8   c_oficina         5889 non-null   int64  
 9   c_CP              0 non-null      float64
 10  c_tipo_asenta     5889 non-null   int64  
 11  c_mnpio           5889 non-null   int64  
 12  id_asenta_cpcons  5889 non-null   int64  
 13  d_zona            5889 non-null   object 
 14  c_cve_ciudad      2120 non-null   float64
dtypes: float64(2), int64(7), object(6)
memory usage: 690.2+ KB


In [11]:
# Obtener el DataFrame específico para el estado 'Puebla'
df_codigos_postales = pd.read_excel(archivo_excel, sheet_name='Puebla', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales['estado'] = 'Puebla'  # Agregar el nombre del estado como 'estado'
df_codigos_postales.head()

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado
0,72000,Centro,Colonia,Puebla,Puebla
1,72000,San Francisco,Colonia,Puebla,Puebla
2,72010,Santa María la Rivera,Colonia,Puebla,Puebla
3,72010,INFONAVIT Bosques 5 de Febrero,Unidad habitacional,Puebla,Puebla
4,72010,Residencial Santa María,Unidad habitacional,Puebla,Puebla


In [12]:
df_codigos_postales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5889 entries, 0 to 5888
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   d_codigo       5889 non-null   int64 
 1   d_asenta       5889 non-null   object
 2   d_tipo_asenta  5889 non-null   object
 3   D_mnpio        5889 non-null   object
 4   estado         5889 non-null   object
dtypes: int64(1), object(4)
memory usage: 230.2+ KB


In [13]:
# Limpiar y estandarizar las columnas
df_codigos_postales['D_mnpio'] = df_codigos_postales['D_mnpio'].str.lower().str.strip()
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].str.lower().str.strip()
df_codigos_postales['d_tipo_asenta'] = df_codigos_postales['d_tipo_asenta'].str.lower().str.strip()
df_codigos_postales['estado'] = df_codigos_postales['estado'].str.lower().str.strip()

In [14]:
df.columns

Index(['id', 'categoria', 'precio', 'precio_mxn', 'precio_usd',
       'fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
       'precio_m2_terreno', 'precio_m2_construido', 'tiempo_de_publicacion',
       'meses_transcurridos', 'meses_transcurridos_fecha', 'tipo', 'Status',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total',
       'seguridad_privada', 'tipo_lugar', 'Colonia', 'Municipio', 'Estado',
       'CP', 'ubicacion', 'url', 'descripcion'],
      dtype='object')

In [16]:
print(df[['tipo_lugar','Colonia','Municipio','Estado','ubicacion']])

     tipo_lugar               Colonia Municipio  Estado  \
0          None         puebla centro    puebla  puebla   
1          None  las hadas mundial 86    puebla  puebla   
2          None    plazas de amalucan    puebla  puebla   
3          None      heroes de puebla    puebla  puebla   
4          None    plazas de amalucan    puebla  puebla   
...         ...                   ...       ...     ...   
3471       None                puebla    puebla  puebla   
3472       None           angelopolis    puebla  puebla   
3473       None                la paz    puebla  puebla   
3474       None    san jose mayorazgo    puebla  puebla   
3475       None      centro historico    puebla  puebla   

                                 ubicacion  
0            Puebla Centro, Puebla, Puebla  
1     Las Hadas Mundial 86, Puebla, Puebla  
2       Plazas de Amalucan, Puebla, Puebla  
3         Héroes de Puebla, Puebla, Puebla  
4       Plazas de Amalucan, Puebla, Puebla  
...                  

In [17]:
# Eliminar acentos de df_codigos_postales para que se pueda mapear bien
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [40]:
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'centro' else x)
#df['Colonia'] = df['Colonia'].apply(lambda x: 'plazas amalucan' if x == 'plazas de amalucan' else x)
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'puebla' else x)
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'puebla centro' else x)

In [41]:
# Mapear colonias de df_codigos_postales a las colonias de df para estandarizar

# Coincidencias exactas
df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()].empty else x)
# Coincidencias parciales
df['Colonia_Estandarizada'] = df['Colonia_Estandarizada'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)

print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']])

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16440\3161061601.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['Colonia_Estandarizada'] = df['Colonia_Estandarizada'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)


     tipo_lugar               Colonia    Colonia_Estandarizada Municipio  \
0          None                centro                   centro    puebla   
1          None  las hadas mundial 86     las hadas mundial 86    puebla   
2          None       plazas amalucan          plazas amalucan    puebla   
3          None      heroes de puebla  los heroes de puebla ii    puebla   
4          None       plazas amalucan          plazas amalucan    puebla   
...         ...                   ...                      ...       ...   
3471       None                centro                   centro    puebla   
3472       None           angelopolis              angelopolis    puebla   
3473       None                la paz                   la paz    puebla   
3474       None    san jose mayorazgo       san jose mayorazgo    puebla   
3475       None                centro                   centro    puebla   

      Estado                             ubicacion  
0     puebla         Puebla Centro

In [46]:
# Función para mapear colonias
def mapear_colonia(colonia, df_codigos_postales):
    # Coincidencia exacta
    coincidencia_exacta = df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == colonia.lower()]
    if not coincidencia_exacta.empty:
        return coincidencia_exacta['d_asenta'].values[0]
    
    # Coincidencia parcial
    coincidencia_parcial = df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(colonia, case=False)]
    if not coincidencia_parcial.empty:
        return coincidencia_parcial['d_asenta'].values[0]
    
    # Si no hay coincidencia, devolver el valor original
    return colonia

# Aplicar el mapeo de colonias en el DataFrame original
df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: mapear_colonia(x, df_codigos_postales))

# Mostrar las columnas relevantes para verificar el resultado
print(df[['tipo_lugar', 'Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']])


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16440\4208812339.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  coincidencia_parcial = df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(colonia, case=False)]


     tipo_lugar               Colonia Colonia_Estandarizada Municipio  Estado  \
0          None                centro                centro    puebla  puebla   
1          None  las hadas mundial 86  las hadas mundial 86    puebla  puebla   
2          None       plazas amalucan       plazas amalucan    puebla  puebla   
3          None      heroes de puebla      heroes de puebla    puebla  puebla   
4          None       plazas amalucan       plazas amalucan    puebla  puebla   
...         ...                   ...                   ...       ...     ...   
3471       None                centro                centro    puebla  puebla   
3472       None           angelopolis           angelopolis    puebla  puebla   
3473       None                la paz                la paz    puebla  puebla   
3474       None    san jose mayorazgo    san jose mayorazgo    puebla  puebla   
3475       None                centro                centro    puebla  puebla   

                           

In [49]:
# Función para obtener el código postal dado estado, municipio, y colonia
def obtener_codigo_postal(colonia, municipio, estado, df_codigos_postales):
    # Normalizar los inputs
    estado = estado.lower().strip()
    municipio = municipio.lower().strip()
    colonia = colonia.lower().strip()
    
    #estado = estado.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    #municipio = municipio.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    #colonia = colonia.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))

    # Filtrar el DataFrame según estado y municipio
    df_filtrado = df_codigos_postales[(df_codigos_postales['estado'] == estado) & (df_codigos_postales['D_mnpio'] == municipio)]
    
    # Intentar encontrar coincidencias exactas de la colonia
    resultado = df_filtrado[df_filtrado['d_asenta'] == colonia]
    
    if not resultado.empty:
        # Si se encuentra una coincidencia, devolver el código postal
        return resultado['d_codigo'].values[0]
    
    return None  # Devolver None si no hay coincidencia


In [50]:
df['codigo_postal'] = df.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal = df[df['codigo_postal'].isna()]
print(sin_codigo_postal[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal.shape)

                             Colonia Municipio  Estado  codigo_postal
17                           la loma    puebla  puebla            NaN
18             santa maria la ribera    puebla  puebla            NaN
28             san diego castillotla    puebla  puebla            NaN
34    san miguel mayorazgo infonavit    puebla  puebla            NaN
35    san miguel mayorazgo infonavit    puebla  puebla            NaN
...                              ...       ...     ...            ...
3446                        el pilar    puebla  puebla            NaN
3450            el mirador la calera    puebla  puebla            NaN
3453          geo villas atlixcayotl    puebla  puebla            NaN
3457          geo villas atlixcayotl    puebla  puebla            NaN
3469               campestre paraiso    puebla  puebla            NaN

[536 rows x 4 columns]
(536, 32)


1213 -> 1124 -> 1063

In [51]:
df.shape

(3476, 32)

In [52]:
3476-516

2960

In [53]:
sin_codigo_postal['Colonia'].value_counts()

Colonia
centro                      36
bosques de san sebastian    32
la loma                     31
lomas de angelopolis        29
bosques amalucan            26
                            ..
la escondida                 1
las fuentes de puebla        1
santa cruz                   1
insurgentes zaragoza         1
el pilar                     1
Name: count, Length: 80, dtype: int64

In [56]:
dataos = df_codigos_postales[df_codigos_postales['d_asenta'].str.contains('centro', case=False)]
dataos['d_asenta'].value_counts()
#df_codigos_postales['d_asenta'].value_counts()

#'centro historico'

d_asenta
centro                              3
centro cruz del sur                 1
las animas centro comercial         1
centro comercial puebla             1
insurgentes centro                  1
cholula de rivadabia centro         1
san andres cholula centro           1
centro comercial palmas plaza       1
amozoc centro                       1
xicotepec de juarez centro          1
huauchinango centro                 1
zacatlan centro                     1
lindavista centro                   1
zacapoaxtla centro                  1
libres centro                       1
teziutlan centro                    1
coatectzin centro                   1
yaonahuac centro                    1
hueyapan centro                     1
teteles de avila castillo centro    1
centro (tepatlaxco)                 1
huejotzingo centro                  1
atlixco centro                      1
izucar de matamoros centro          1
ahuatlan centro                     1
jolalpan centro                     1
san

In [55]:
centro_puebla = dataos[(dataos['d_asenta'] == 'centro') & (dataos['D_mnpio'].str.contains('Puebla', case=False))]
centro_puebla

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado


In [50]:
print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(35))

   tipo_lugar                         Colonia           Colonia_Estandarizada  \
0        None                   puebla centro                   puebla centro   
1        None            las hadas mundial 86            las hadas mundial 86   
2        None                 plazas amalucan                 plazas amalucan   
3        None                heroes de puebla         los heroes de puebla ii   
4        None                 plazas amalucan                 plazas amalucan   
5        None              san jose mayorazgo              san jose mayorazgo   
6        None           ex-hacienda mayorazgo           ex-hacienda mayorazgo   
7        None                   puebla centro                   puebla centro   
8        None                       la calera           plan de ayala caleras   
9        None                       el carmen                nueva del carmen   
10       None                     santa maria           santa maria la rivera   
11       None               

In [10]:
from rapidfuzz import process

#Lista de colonias estandarizadas
colonias_estandar = ['centro historico','colonia 2']

#Aplicar fuzzy matching para corregir por similitud((
df['Colonia'] = df['Colonia'].apply(lambda x: process.extractOne(x, colonias_estandar)[0])
df['Colonia'].unique()


centro historico

In [54]:
print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(35))

   tipo_lugar                         Colonia           Colonia_Estandarizada  \
0        None                   puebla centro                   puebla centro   
1        None            las hadas mundial 86            las hadas mundial 86   
2        None              plazas de amalucan              plazas de amalucan   
3        None                heroes de puebla                heroes de puebla   
4        None              plazas de amalucan              plazas de amalucan   
5        None              san jose mayorazgo              san jose mayorazgo   
6        None           ex-hacienda mayorazgo           ex-hacienda mayorazgo   
7        None                   puebla centro                   puebla centro   
8        None                       la calera           plan de ayala caleras   
9        None                       el carmen                nueva del carmen   
10       None                     santa maria                     santa maria   
11       None               